In [1]:
import colorsys #это для преобразования #ff00ff в формат rgb (255, 00, 255)

from PIL import Image, ImageFilter

#####       Вводим нужные пременные: нужный цвет и fizziness 


# Цвет, который нужно выделить в шестнадатеричном формате
hex_color = "#282c34"

# Определите допустимую разницу в цвете между пикселем и target_color
# (Fuzziness: 168 в Photoshop)
fuzziness = 180


In [2]:
hex_color = hex_color[1:] # Удалите символ "#" из строки

rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) # Преобразуйте шестнадцатеричный цвет в RGB

rgb_color = tuple(c / 255.0 for c in rgb_color) # Преобразуйте RGB в диапазоне 0-255 в диапазон 0-1

hls_color = colorsys.rgb_to_hls(*rgb_color[:3]) # Преобразуйте RGB в формат HLS


rgb_color = tuple(round(c * 255) for c in colorsys.hls_to_rgb(*hls_color)) # Преобразуйте HLS в формат RGB

target_color = (40, 44, 52)

In [3]:
#Устанавливаем Pillow если его нету
!pip install Pillow


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os

In [5]:
# Создайте функцию-ламбду, которая будет заменять пиксели нужного цвета на прозрачный цвет
def remove_color(pixel):
    r, g, b = pixel
    if (r, g, b) == target_color:
        return (0, 0, 0, 0)
    else:
        return (r, g, b, 255)


for file_name in os.listdir("Cropped"):
     if file_name.endswith(".jpg") or file_name.endswith(".png"):
         # Откройте изображение
         img = Image.open(os.path.join("Cropped", file_name))
         img = img.convert("RGBA")
         img = img.point(remove_color)
         img.save(os.path.join("Cropped", "removed_bg_" + file_name))


TypeError: cannot unpack non-iterable int object

In [6]:

# Переберите все файлы в папке Croped
for file_name in os.listdir("Cropped"):
    if file_name.endswith(".jpg") or file_name.endswith(".png"):
        # Откройте изображение
        img = Image.open(os.path.join("Cropped", file_name))
        
        # Создайте маску, содержащую пиксели, близкие по цвету к target_color
        mask = img.convert("L").point(lambda pixel: 255 if abs(pixel - target_color[0]) + abs(pixel - target_color[1]) + abs(pixel - target_color[2]) <= fuzziness else 0)

        # Размыть маску для сглаживания границ
        mask = mask.filter(ImageFilter.GaussianBlur(5))
        
        # Примените маску к изображению и сохраните результат
        img.putalpha(mask)
        img.save(os.path.join("Cropped_no_bg", "removed_bg_" + file_name))